## 5.2 Data Preprocessing for Tensorflow

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import random

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

import json

## Prepro for Training Set

In [ ]:
MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 25

DATA_IN_PATH = './data_in/'
TRAIN_Q1_DATA = 'train_q1.npy'
TRAIN_Q2_DATA = 'train_q2.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
NB_WORDS_DATA = 'nb_words.json'

In [ ]:
DEFAULT_PATH = '~/.kaggle/competitions/quora-question-pairs/'

In [ ]:
train_df = pd.read_csv(DEFAULT_PATH + 'train.csv', encoding='utf-8')
train_df = train_df.dropna() 

#### neg 데이터 Data balance 맞추기

In [ ]:
train_pos_df = train_df.loc[train_df['is_duplicate'] == 1]
train_neg_df = train_df.loc[train_df['is_duplicate'] == 0]

class_diff_count = train_neg_df.shape[0] - train_pos_df.shape[0]
sample_frac = class_diff_count / train_neg_df.shape[0]

train_neg_df = train_neg_df.sample(frac=sample_frac)

train_df = pd.concat([train_neg_df, train_pos_df])

In [ ]:
train_df.count()

In [ ]:
FILTERS = "([~.,!?\"':;)(])"

changeFilter = re.compile(FILTERS)

questions = list(train_df['question1']) + list(train_df['question2'])
filtered_questions = list()
for q in questions:
     filtered_questions.append(re.sub(changeFilter, "", q).lower())

In [ ]:
filtered_questions[0]

In [ ]:
train_df.count()

In [ ]:
questions = list(train_df['question1']) + list(train_df['question2'])
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(questions)
word_index = tokenizer.word_index

question1_word_sequences = tokenizer.texts_to_sequences(train_df['question1'])
question2_word_sequences = tokenizer.texts_to_sequences(train_df['question2'])

print("Words in index: %d" % len(word_index))

In [ ]:
question1_word_sequences[0]

In [ ]:
is_duplicate = train_df['is_duplicate']

In [ ]:
q1_data = pad_sequences(question1_word_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
q2_data = pad_sequences(question2_word_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
labels = np.array(is_duplicate, dtype=int)
print('Shape of question1 data tensor:', q1_data.shape)
print('Shape of question2 data tensor:', q2_data.shape)
print('Shape of label tensor:', labels.shape)

In [ ]:
data_prepro_configs = {'vocab': tokenizer.word_index, 'vocab_size': len(tokenizer.word_counts)}

In [ ]:
if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)

np.save(open(DATA_IN_PATH + TRAIN_Q1_DATA, 'wb'), q1_data)
np.save(open(DATA_IN_PATH + TRAIN_Q2_DATA, 'wb'), q2_data)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), labels)

In [ ]:
with open(DATA_IN_PATH + NB_WORDS_DATA, 'w') as f:
    json.dump(data_prepro_configs, f)

## Prepro for Test Set

In [ ]:
TEST_Q1_DATA = 'test_q1.npy'
TEST_Q2_DATA = 'test_q2.npy'

In [ ]:
test_df = pd.read_csv(DEFAULT_PATH + 'test.csv', encoding='utf-8')
test_df = test_df.dropna() 

In [ ]:
test_df.head(5)

In [ ]:
test_question1_word_sequences = tokenizer.texts_to_sequences(test_df['question1'])
test_question2_word_sequences = tokenizer.texts_to_sequences(test_df['question2'])

In [ ]:
test_q1_data = pad_sequences(test_question1_word_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
test_q2_data = pad_sequences(test_question2_word_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [ ]:
print('Shape of test question1 data tensor:', test_q1_data.shape)
print('Shape of test question2 data tensor:', test_q2_data.shape)

In [ ]:
np.save(open(DATA_IN_PATH + TEST_Q1_DATA, 'wb'), test_q1_data)
np.save(open(DATA_IN_PATH + TEST_Q2_DATA, 'wb'), test_q2_data)